# Python Process Book for CS109 Project


## Overview and Motivation:

This project looks at trends in health topics over time.  The members of this project team are all public health students.  We were interested in how topics in health in a popular newspaper, The New York Times, have changed over time.  We decided to gain a deeper understanding of Latent Dirilecht Allocation (LDA) and use this topic modeling method to find the major topics in health over the past 5 decades (the period of time we were able to collect data for).  

Related work which inspired our project was Homework 5 for this class as well as Google Trends which show trends in Google searches over time for specific topics.

## Initial Questions:

 Which topics are persistent over time?  
 
 Which topics have a spike, when do they occur, and why did it happen? 
 Are there any surprising topics?
 
 With regards to LDA:
 Should we use Pattern (as in Hw5) or is there a better option for our data?
 What hyperparameters should we choose?
 
 Are there further analyses we can consider based on our findings?

## Data:

Data was pulled from the New York Times article API.  Using the API console (http://developer.nytimes.com/io-docs), we were able to inspect the type of results for a given query.  Originally, we decided to look at results using the 'fq=newsdesk:Health' option which would pull results under the Health topic section of the Times (approximately 680,000 documents since 1851).  However, after looking at the first 1000 results, we found that the documents pulled mainly consisted of videos, slideshows, and interactive features instead of articles.  We remedied this issue by instead using a query for the keyword 'Health' which searched all articles and their headlines for the word health.  Looking at the results from the API console showed that overall, using 'Health' as our query term instead of newsdesk:Health produced approiximately 40,000 more documents (720,000 total).

The Times API has several limiting factors when pulling data: 10,000 calls per day and a maximum of 100 pages per query.  To handle these limitations, our code pulled data by year and split each month into 3 parts (based on testing date ranges for number of pages which would be pulled so the total number of pages would be less than 100).  Year and count were entered manually to keep track of how many calls were being made and to break up the data calls for when errors occurred (such as timeouts, key errors, and date errors; when these errors were encountered the solution was appended to the code which resulted in the final version below).     

In this code, we requested the json dictionaries, used the relevant information to create a dataframe with the date, id, document type (article, blog, video), newsdesk section and subsection, and text from the title, abstract, and first paragraph.  The data was saved in a csv file for that section and tracked using an excel file - [DateTracker](https://github.com/dyan1211/teamsignificant/blob/master/DateTracker.xlsx).

At the beginning, we wanted to pull as much data as possible to get an idea of topic changes over time - in particular if there was a difference between today and some unspecified earlier time (articles range back to the mid-1800s).  Due to the limiting factors (and keeping in mind amount of time needed for analysis), we pulled data through the late 1950s.  We decided to perform our analysis on data from the end of October 2015 through 1966 the end of December 1966, giving us five decades of data.  

Text scraping and cleaning was done in https://github.com/dyan1211/teamsignificant/blob/master/DataScraping.ipynb

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [ ]:
import json

In [ ]:
%%time
count = 1460 #enter new count starting number
year =  1954 #enter year
months = ['01', '01', '01', '02', '02', '02', '03', '03', '03', '04', '04', '04', '05', '05', '05', '06', '06', '06', '07', '07', '07', '08', '08', '08', '09', '09', '09', '10', '10', '10', '11', '11', '11', '12', '12', '12']
startdays = ['01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21']
enddays = ['10', '20', '31', '10', '20', '27', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31']

pcount = 0

for d in range(36):
    sdate = year + months[d] + startdays[d]
    edate = year + months[d] + enddays[d]

    docs=[]
    #get 1st page and number of documents
    url1 = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=1&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(sdate, edate) 
    lpage = requests.get(url1).json()['response']['meta']['hits']
    
    #calculate number of pages for call (divide total documents by 10 and add 2 in case not every page has 10 documents); 
    #if there are no hits skip to end
    if lpage is not 0:
        numpages = int(lpage/10 + 2) 
        pcount += numpages
    
        #get json files for first page
        pagedoc1 = requests.get(url1).json()['response']['docs']
        for j in range(0,len(pagedoc1)):
            docs.append(pagedoc1[j])    
    
        #get json dictionaries for rest of the pages
        for i in range(2, numpages): 
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page={}&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(i, sdate, edate)
            pagedocs = requests.get(url).json()['response']['docs']
            time.sleep(1)
        
            for j in range(0,len(pagedocs)):
                docs.append(pagedocs[j])

    #pull information from json file into dictionary        
        docsinfo = []
        for d in docs:
            obs = {}
            obs['id'] = d['_id']
            obs['type'] = d['type_of_material']
            obs['doctype'] = d['document_type']
            obs['date'] = d['pub_date']
            obs['news_desk'] = d['news_desk']
            obs['section'] = d['section_name']
            obs['subsection'] = d['subsection_name']
            obs['abstract'] = d['abstract']
            obs['paragraph'] = d['lead_paragraph']
        
            #Headline exceptions
            if d['headline'].get('main') is not None:
                obs['headline'] = d['headline']['main']
            elif d['headline'].get('name') is not None:
                obs['headline'] = d['headline']['name']
            else:
                obs['headline'] = ' '
    
            #get the date part of datetime
            if obs['date'] is not None:
                obs['date'] = obs['date'][0:10]
    
            #Remove empty abstract and lead paragraph cell to join text
            if obs['abstract'] is None:
                a = ' '
            else: 
                a = obs['abstract']
            if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
                p = ' '
            else:
                p = obs['paragraph']
    
            #Join all the text columns
            text = [obs['headline'], p, a]
            obs['text'] = " ".join(text)
    
            docsinfo.append(obs)

    #create dataframe from dictionary, make date column date type, and store in csv
        docsdf = pd.DataFrame(docsinfo)
        docsdf['date'] = pd.to_datetime(docsdf['date'])
        docsdf.to_csv("data/docsdf-{}.csv".format(count), encoding = 'utf-8') 

    count += 1 
    

Next the files were concatenated into a single dataframe and saved to a csv file of our data which is located in our dropbox: https://www.dropbox.com/s/eedgwugamiw0zwd/total.csv?dl=0 

In [ ]:
#1966-2015
#There were no documents for 08/11/1978 - 10/31/1978 (leading to gap in csv files)
frames = []
for i in range(1,1338) : 
    dfs = pd.read_csv("docsdf-"+str(i)+".csv")
    frames.append(dfs)
for i in range(1346,1784) : 
    dfs = pd.read_csv("docsdf-"+str(i)+".csv")
    frames.append(dfs)
totaldf = pd.concat(frames)
totaldf['date'] = pd.to_datetime(totaldf['date'])

In [ ]:
totaldf.to_csv("total.csv", index=False)

We looked at the types of documents to see if there were any trends and found that the majority of the documents were articles.  We are mainly interested in articles, and two of the document types are more recent types (multimedia and blogpost), so we decided to only include articles in our corpus.

Document analysis was done in https://github.com/dyan1211/teamsignificant/blob/master/Exploration.ipynb

In [ ]:
# Code to look at frequencies of document types
sns.set(style="white", context="talk")
ax = sns.barplot(x=('Article', 'Blogpost', 'Column', 'Multimedia', 'Recipe'), y=type_counts)
ax.set(title="Document Type Frequencies",ylim=(0,400000),yticks=[100000,200000,300000,400000])
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+0.2, height+10000, '%d'%height, fontsize=14)
#ax.text(-0.2,380000, "369,890", fontsize=14)
#ax.text(0.82,40000, "31,369", fontsize=14)
sns.despine(bottom=True)

![](http://i.imgur.com/qLMXD3e.png)

In [ ]:
#Only use documents that were specified as article will be used
df = totaldf[totaldf['doctype'] == 'article']

After plotting our data over time and checking days which had highest number of articles, we found that the years 1980 and 2014 had some strange spikes.  We realized there were a number of duplicate documents in our data which had pieces of another documents (such as a shorted title or only the first few sentences of a paragraph). We utilized the drop_duplicates function to by creating a new column which contained the first 50 characters of the paragraph column in order to capture the matching sections of the two documents.

Duplicate dropping was done in https://github.com/dyan1211/teamsignificant/blob/master/DuplicateArticles.ipynb

In [ ]:
#Bar graph of number of documents per year before dropping duplicates (spike in 1980 and 2014)
years = df.year.value_counts().sort_index()
years.plot(kind='bar')

![](http://imgur.com/rVyNJx1.png)

In [ ]:
#Check dates with most articles 
days = articledf.date.value_counts()
days[:20].plot(kind='barh');

![](http://imgur.com/caAkxtg.png)

In [ ]:
#Remove duplicates
df['dupcheck'] = df['paragraph'].str[0:50]
df = df.drop_duplicates('dupcheck')

In [ ]:
#Bar graph of number of documents per year after dropping duplicates (smoothed out 1980, though still 2014 spike )
years = df.year.value_counts().sort_index()
years.plot(kind='bar')

![](http://imgur.com/dHtCfzN.png)

This graph looks much better though there is still the 2014 spike.  Using the API console through NYT, I check the total number of articles in 2014 compared to 2013 (not just 'Health' articles):

Total # of articles produced by NYT in 2013 = 117593

Total # of articles produced by NYT in 2014 = 308867

So the spike is not specific to our search.  We do not have a concrete explanation, though we considered the possibility that it is due to an increase in web content.

#### Choice of Time Periods

Before moving forward, we made the decision that in order to parse out clear and specific topics we should perform LDA on subsets of the data.  By using the entire dataset, we were concerned that the topics would end up being too general.  We decided to use 5 year periods - large enough to have data to run LDA on and to capture topics which would have a trend in time, but small enough that some major events would show up. 

The code for splitting the df was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

## Exploratory Data Analysis:

We used spark to clean and analyze our data since these processes are easily parallelized.  Spark was implemented using homebrew on a Mac. 

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = '/Applications/anaconda/bin/python'

In [ ]:
import findspark
findspark.init()
print findspark.find()

In [ ]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
import sys
rdd = sc.parallelize(xrange(10),10)

In [ ]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

We began by cleaning our code in order to perform LDA.  We decided to use NLTK to process our text since it is the leading platform for natural language processing.  However, we found that NLTK takes an extremely long time to process our large data set.  Instead, we utilized TextBlob which works off of both NLTK and Pattern platforms.  TextBlob also has the advantage of implementing the Averaged Perceptron algorithm which has been shown to be faster and more accurate than NLTK and Pattern (http://stevenloria.com/tutorial-state-of-the-art-part-of-speech-tagging-in-textblob/).

The text was tokenized, tagged for part of speech, made all lowercase, lemmatized, and the nouns were extracted.  

We considered issues with making words all lowercase due to some nouns such as AIDs and WHO but decided that not using .lower would potentially cause more problems.  In addition, the AP tagger should be able to identify nouns based on the sentence structure and even if a word has multiple meanings, if that word is mostly used in a specific way (HIV/AIDs vs government aid), it should not be difficult to identify based on the other words which make up the topic and the documents associated with it.  (In fact, we were able to distinguish between the two in our analysis).

The initial NLTK work was done in >>>>>>

TextBlob work (and the following parseout work) was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

In [ ]:
from textblob import TextBlob as tb
from textblob_aptagger import PerceptronTagger
from textblob import Blobber

In [ ]:
import nltk
from nltk.corpus import stopwords #stopwords

In [ ]:
#The necessary NLTK packages need to be downloaded to implement some of the functions. RUN ONLY ONCE
nltk.download()

In [ ]:
#Set up Averaged Perceptron Tagger for TextBlob
tb = Blobber(pos_tagger=PerceptronTagger())

In [ ]:
#Clean the text for each document to extract the nouns 
#Cleaning includes lemmatization and removal of stopwords

def get_parts(thetext):
    nouns=[]
    tagged = tb(thetext).tags # a list of tuples
    for tup in tagged:
        w, tag = tup  
        if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
            word = w.lemmatize().lower()
            if word[-1] in punctuation : 
                word = word[:-1]
            if word in stops or word in punctuation or len(word)==1 :
                continue
            nouns.append(word)
    nouns2=[]
    for n in nouns:
        if len(n)!=0:
            nouns2.append(n)
    return nouns2

Parse out the nouns and organize in a list for each document.  We decided to parse on the document level (instead of the sentence level) under the assumption that each document could be classified as a topic (or a few topics).

*This code was run separately for each 5 year period - all example output is for 1966

In [ ]:
%%time

parseout = []
for index, row in df.iterrows() : 
    parseout.append(get_parts(row.text))

In [ ]:
# saving parseout to txt file
with open("parseout.txt", 'w') as f:
    json.dump(parseout,f)

Using the output for each 5-year time period, we were able to create word clouds to get an idea of the words in our corpus.  Not surprisingly, we can see that generalized words dominate - new, city, state, today.  But we can also pick out the presidents during most periods and some health related words also jump out - program, hospital, doctor. 

Wordcloud work was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

Code is based on https://github.com/amueller/word_cloud

In [ ]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS

In [ ]:
# read the mask image taken from http://static01.nyt.com/images/icons/t_logo_291_black.png
nyt_mask = np.array(Image.open("~/nyt_mask_2.png"))

In [ ]:
wc = WordCloud(background_color="white", max_words=2000, mask=nyt_mask, stopwords=STOPWORDS)

In [ ]:
# create single string of all text for each parseout list
import itertools
document1966 = list(itertools.chain.from_iterable(parseout1966))
document1966 = ' '.join(document1966)
wc.generate(document1966)  #generate wordcloud

# show
plt.imshow(wc)
plt.axis("off")
plt.figure()
plt.show()

![](http://i.imgur.com/m7nfPPr.png)

### LDA 

LDA was done in >>>>>>

We started by created a corpus of our words.

In [ ]:
from gensim import corpora, models, similarities, matutils

In [ ]:
dictionary = corpora.Dictionary(documents)
dictionary.filter_extremes(no_below=5,no_above=0.75,keep_n=100000)
dictionary.compactify()

In [ ]:
corpus = [dictionary.doc2bow(document) for document in documents]

We wanted a way to decide how many topics would be best for our data.  In addition, since each of our 5-year time periods had a different number of documents, we assumed they were likely to also have a different number of topics (or the corpus for that period may be able to support more topics based on its content). Ideally, we would check the topic output after using several different options for number of topics and decide which worked best for our data.  Due to time constraints, we searched for an automatic method. After searching for different ways to approach this task, we found a method to find the number of topics using  KL divergence based on the idea that we can view LDA as a matrix factorization.  

Code for this section (and details for the method) are based on: http://blog.cigrainger.com/2014/07/lda-number.html

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def sym_kl(p,q):
    return np.sum([stats.entropy(p,q),stats.entropy(q,p)])

In [ ]:
def arun(corpus,dictionary,min_topics,max_topics,step):
    kl = []
    for i in range(min_topics,max_topics,step):
        lda = models.ldamodel.LdaModel(corpus=corpus,
            id2word=dictionary,chunksize=340,num_topics=i)  #we decided on a chunksize of approximately .01 of number of documents
        m1 = lda.expElogbeta
        U,cm1,V = np.linalg.svd(m1)
        #Document-topic matrix
        lda_topics = lda[corpus]
        m2 = matutils.corpus2dense(lda_topics, lda.num_topics).transpose()
        cm2 = l.dot(m2)
        cm2 = cm2 + 0.0001
        cm2norm = np.linalg.norm(l)
        cm2 = cm2/cm2norm
        kl.append(sym_kl(cm1,cm2))
    return kl

In [ ]:
l = np.array([sum(cnt for _, cnt in doc) for doc in corpus])
kl = arun(corpus,dictionary,min_topics=1,max_topics=100,step=5)  #searched best number of topics between 1 and 100 by 5

In [ ]:
# Plot kl divergence against number of topics
x = np.arange(1,100,5)
plt.plot(x, kl)
plt.ylabel('Symmetric KL Divergence')
plt.xlabel('Number of Topics')
plt.savefig('kldiv1986.png', bbox_inches='tight')

![](http://i.imgur.com/9aRvg9m.png)

For the LDA we also needed to decide on the number for chunksize (the number of documents which are loaded into memory at a time).  Using 1966 as an example (with number of topics equal to 50), we tested chunksizes of 10%, 5%, and 1% of the total document number.  After inspecting the topic output, we found that smaller chunksizes (1% of document total) provided clearer topics (the grouped words told a clearer story on average for this chunksize).  

Therefore, for each period, we ran LDA using the number of topics chosen from above and a chunksize of 1% of document size.  

The output from our LDA (lda.print_topics) can be found [here](https://docs.google.com/a/mail.harvard.edu/document/d/12jsDy1T5_7QjslxpsDnzfzH60H00rV0YEPizmwzknNU/edit?usp=sharing)

We had a total of 520 topics - 50 topics on average for each period.

In [ ]:
import gensim

In [ ]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary,chunksize=220, num_topics=50)

In [ ]:
lda.print_topics(num_topics=50)

#### Creating new data frame 

We appended to each 5-year dataframe a column for each topic.  The data in these columns was the percent of the document attributed to that topic.

Checking topics - we looked at the top 10-20 articles with the highest percent for a topic to see the specific articles which fell under the topic (useful for unclear topics and identifying specific events).

In [ ]:
import heapq

In [ ]:
#Get the top 20 articles with the highest percent for a topic to 
top20 = [list(fulldf['Topic1']).index(p) for p in heapq.nlargest(20, list(fulldf['Topic1']))]
list(fulldf['text'][top20])[0:19]

## Final Analysis:

## Presentation: